### Задача:
Сгенерировать DataFrame такого размера, который гарантированно не влезет в Storage Memory (например, 10 млн строк с длинным текстом при памяти 512mb).

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
import time

# Даем экзекьютору всего 512 мб памяти
spark = SparkSession.builder \
        .appName("Lab_02_03") \
        .master("local[*]") \
        .config("spark.driver.memory", "512m") \
        .config("spark.executor.memory", "512m") \
        .getOrCreate()

In [2]:
# Создаем "тяжелый" DataFrame (около 200-300 МБ в памяти)
# Генерируем 5 млн строк с длинной строкой
df = spark.range(0, 100000000).withColumn(
    "dummy_data",
    lit("x" * 1000)
)

# Помечаем, что хотим положить его в RAM (Storage Memory)
df.cache()

# Внимание! .cache() - ленивая операция. Данные попадут в память только при Action.
print("Начинаем кэширование...")
count = df.count() # # Action вызывает чтение и запись в кэш
print(f"Кэшировано {count} строк")

Начинаем кэширование...
Кэшировано 100000000 строк


#### Зафиксировать в Spark UI (вкладка Storage), какой % данных влез в память.
#### Зафиксировать, куда делись остальные данные (Disk?). Подсказка: Вкладка Storage показывает "Size on Disk".